# Imports

In [1]:
# Numerai API
from numerapi import NumerAPI

# data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# machine learning models
from sklearn.linear_model import LinearRegression

# other
import json
from tqdm import tqdm

# my utils
from utils import *

# Download data

In [2]:
napi = NumerAPI()
round = napi.get_current_round()
era = round + 695

napi.download_dataset('v4/features.json', 'data/features.json')
napi.download_dataset('v4/train_int8.parquet', 'data/train.parquet')
napi.download_dataset('v4/validation_int8.parquet', 'data/validation.parquet')
napi.download_dataset('v4/live_int8.parquet', f'data/live_{round}.parquet')

2022-08-03 11:16:51,368 INFO numerapi.utils: target file already exists
2022-08-03 11:16:51,371 INFO numerapi.utils: download complete
2022-08-03 11:16:53,034 INFO numerapi.utils: target file already exists
2022-08-03 11:16:53,038 INFO numerapi.utils: download complete
2022-08-03 11:16:54,101 INFO numerapi.utils: target file already exists
2022-08-03 11:16:54,104 INFO numerapi.utils: download complete
2022-08-03 11:16:55,163 INFO numerapi.utils: target file already exists
2022-08-03 11:16:55,165 INFO numerapi.utils: download complete


# Features

In [3]:
COLUMNS = [ERA, DATA] + X_COLS + [Y_TRUE]
ITC = 'intercept'
COEFS = X_COLS + [ITC]

# Dataframes

In [4]:
df = pd.read_parquet('data/train.parquet', columns=COLUMNS)
df[ERA] = df[ERA].astype('int32')

# Compute coefs of linear regression by era

In [5]:
def coefs_linreg(df):
    model = LinearRegression()
    model.fit(df[X_COLS], df[Y_TRUE])
    y_prd = pd.Series(model.predict(df[X_COLS]))
    y_rnk = y_prd.rank(pct=True)
    ncorr = np.corrcoef(df[Y_TRUE], y_rnk)[0, 1]
    coefs = {X_COLS[i]: [model.coef_[i]] for i in range(model.n_features_in_)}
    coefs[ITC] = [model.intercept_]
    coefs['corr'] = [ncorr]
    coefs = pd.DataFrame(coefs)
    return coefs

In [6]:
df_coefs = df.groupby(ERA).apply(coefs_linreg)
df_coefs[ERA] = np.arange(len(df_coefs)) + 1

# Do a linear regression to predict the coefs as a function of the era

In [7]:
X = df_coefs[ERA].to_numpy().reshape(-1, 1)
y = df_coefs[COEFS].to_numpy()

coef_predictor = LinearRegression()
coef_predictor.fit(X, y)

LinearRegression()

In [8]:
pred_coefs = ['pred_' + c for c in COEFS]
df_coefs[pred_coefs] = coef_predictor.predict(X)

c:\Users\Miguel\anaconda3\lib\site-packages\pandas\core\frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


# Predict coefficients for current era. Make final predictions

In [9]:
coef_predictions = coef_predictor.predict(np.array([[era]]))
w = np.array(coef_predictions[0][0:-1])
b = coef_predictions[0][-1]

In [10]:
df_liv = pd.read_parquet(f'data/live_{round}.parquet', columns=COLUMNS)
df_liv[Y_PRED] = df_liv[X_COLS] @ w + b

# Plots for coefs as a function of the era

In [11]:
for c in tqdm(COEFS):
    fig, ax = plt.subplots()
    ax.plot(df_coefs[ERA], df_coefs[c], label='coefs')
    ax.plot(df_coefs[ERA], df_coefs['pred_' + c], label='linreg')
    ax.set_xlabel('era')
    ax.set_ylabel('coef')
    ax.set_title(f'coef for {c} as function of era')
    ax.legend()

    fig.savefig(f'model-1/figures/{c}.png')
    plt.close(fig)

100%|██████████| 1192/1192 [01:18<00:00, 15.13it/s]
